In [ ]:
from source.class_yolo_trainer import YOLOTrainer

yolo_trainer = YOLOTrainer
yolo_trainer = YOLOTrainer()

In [ ]:
yolo_trainer.image_folder = "pelotas_19-08/images"
yolo_trainer.annotations_folder = "pelotas_19-08/labels"
yolo_trainer.yolo_Classes = "Pellet"
yolo_trainer.test_percentual_divisor = 20
yolo_trainer.slicing()

In [2]:
yolo_trainer.img_sz = 960
yolo_trainer.training_epochs = 20
yolo_trainer.task = "detect"
yolo_trainer.training()

In [ ]:
yolo_trainer.train_model = "train16"
yolo_trainer.object_to_predict = "D:/Judson_projetos/Yolo_trainer/vid_1.mp4"
yolo_trainer.predict_confidence = 0.2
yolo_trainer.predict()

In [ ]:
from utils.check_file import check_file

check_file("any_file")

In [ ]:
from utils.torch_is_available import torch_is_available

torch_is_available()

In [ ]:
from utils.cvat_dataset import cvat_dataset

cvat_dataset("pelotas_19-08", "pelotas_19-08")